<a href="https://colab.research.google.com/github/tg-bomze/collection-of-notebooks/blob/master/Swap_space_and_time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---

# Swap space and time

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Upload video</font></b>
import os
import cv2
import glob
import math
import numpy as np
from tqdm import tqdm
from IPython.display import clear_output

path_to_frames = '/content/frames'
path_to_result = '/content/result'
video_path = '/content/1_0_moose_hw250_fps25.mp4' #@param {type:"string"}

!rm -rf $path_to_frames
!mkdir $path_to_frames
!ffmpeg -y -i $video_path /content/frames/%7d.png -hide_banner -loglevel error
fps_of_video = round(cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FPS))

<a href=""><img src='https://icon-library.com/images/3d-rotation-icon/3d-rotation-icon-3.jpg' width=300></a> 

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Swap space and time</font></b>
rotation = "Z" #@param ["X", "Z"]
reverse = False #@param {type:"boolean"}
!rm -rf $path_to_result
!mkdir $path_to_result
paths = sorted(glob.glob(f'{path_to_frames}/*.*g'))
if len(paths)%2: paths = paths[:-1]
frames = [cv2.imread(i) for i in tqdm(paths)]
video = np.array(frames)
f, h, w = video.shape[:3]
print('--- Input ---')
print(f'Frames: {f}, Height: {h}, Width: {w}\n')
orig_h, orig_w = cv2.imread(f'{path_to_frames}/0000001.png').shape[:2]

if rotation == 'Z': new_video = np.swapaxes(video, 0, 2)
else: new_video = np.swapaxes(video, 0, 1)

for i in tqdm(new_video):
  if not reverse: name = str(len(glob.glob(f'{path_to_result}/*.*g'))).zfill(7)
  else: name = str(len(new_video)-len(glob.glob(f'{path_to_result}/*.*g'))-1).zfill(7)
  cv2.imwrite(f'{path_to_result}/{name}.png', i)

f, h, w = new_video.shape[:3]
print('--- Output ---')
print(f'Frames: {f}, Height: {h}, Width: {w}')
!ffmpeg -y -r $fps_of_video -i /content/result/%07d.png -c:v libx264 -vf fps=$fps_of_video -pix_fmt yuv420p /content/result_z.mp4 -hide_banner -loglevel error

100%|██████████| 250/250 [00:00<00:00, 511.14it/s]


--- Input ---
Frames: 250, Height: 250, Width: 250



100%|██████████| 250/250 [00:01<00:00, 155.29it/s]


--- Output ---
Frames: 250, Height: 250, Width: 250


In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Download result</font></b>
from google.colab import files
files.download('/content/result_z.mp4')



---

# RIFE (upscale)

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Install</font></b>
from IPython.display import clear_output
!pip install sk-video moviepy
%cd /content
!rm -rf /content/arXiv2020-RIFE/
!git clone https://github.com/hzwer/Practical-RIFE arXiv2020-RIFE
%cd /content/arXiv2020-RIFE/
!mkdir /content/arXiv2020-RIFE/train_log
!pip install -U gdown
!set -exo
!gdown --id 1CPJOzo2CHr8AN3GQCGKOKMVXIdt1RBR1
%cd /content/arXiv2020-RIFE/train_log
!7z e /content/arXiv2020-RIFE/RIFE_trained_model_v4.1.zip
if os.path.isfile('/content/arXiv2020-RIFE/RIFE_trained_model_v4.1.zip'):
  clear_output()
  print('Done!')
else: print("Weights didn't download")

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Upload video</font></b>
import os
import cv2
import glob
import math
import numpy as np
from tqdm import tqdm

path_to_frames = '/content/frames'
path_to_result = '/content/result'
video_path = '/content/1_0_moose_hw250_fps25.mp4' #@param {type:"string"}

!rm -rf $path_to_frames
!mkdir $path_to_frames
!ffmpeg -y -i $video_path /content/frames/%7d.png -hide_banner -loglevel error
fps_of_video = round(cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FPS))

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Swap space and time (Z-axis)</font></b>
!rm -rf $path_to_result
!mkdir $path_to_result
paths = sorted(glob.glob(f'{path_to_frames}/*.*g'))
if len(paths)%2: paths = paths[:-1]
frames = [cv2.imread(i) for i in tqdm(paths)]
video = np.array(frames)
f, h, w = video.shape[:3]
print('--- Input ---')
print(f'Frames: {f}, Height: {h}, Width: {w}\n')
orig_h, orig_w = cv2.imread(f'{path_to_frames}/0000001.png').shape[:2]

new_video = np.swapaxes(video, 0, 2)

for i in tqdm(new_video):
  name = str(len(glob.glob(f'{path_to_result}/*.*g'))).zfill(7)
  cv2.imwrite(f'{path_to_result}/{name}.png', i)

f, h, w = new_video.shape[:3]
print('--- Output ---')
print(f'Frames: {f}, Height: {h}, Width: {w}')
!ffmpeg -y -r $fps_of_video -i /content/result/%07d.png -c:v libx264 -vf fps=$fps_of_video -pix_fmt yuv420p /content/result_z.mp4 -hide_banner -loglevel error

In [ ]:
]#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Upscale width</font></b>
frames_of_video = round(cv2.VideoCapture("/content/result_z.mp4").get(cv2.CAP_PROP_FRAME_COUNT))
print('--- Input ---')
print(f'Frames (width): {frames_of_video}\n')

w_mult = "4" #@param ["2", "4"]
exp = str(int(math.log(int(w_mult), 2)))
path_to_output = '/content/rife_w.mp4'

%cd /content/arXiv2020-RIFE/
!python3 inference_video.py --exp=$exp --video /content/result_z.mp4 --output $path_to_output --fps=$fps_of_video --scale=1 --skip

!rm -rf /content/frames
!mkdir /content/frames
!ffmpeg -y -i /content/rife_w.mp4 /content/frames/%7d.png -hide_banner -loglevel error

f = int(w_mult)*orig_w
print('\n--- Output ---')
print(f'Frames (width): {f}')

--- Input ---
Frames (width): 250

/content/arXiv2020-RIFE
skip flag is abandoned, please refer to issue #207.
Loaded 3.x/4.x HD model.
/content/result_z.mp4, 250.0 frames in total, 25.0FPS to 25FPS
Will not merge audio because using png or fps flag!
100% 249/250.0 [00:06<00:00, 40.08it/s]

--- Output ---
Frames (width): 1000


In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Swap space and time (Z-axis back, X-axis)</font></b>
!rm -rf $path_to_result
!mkdir $path_to_result
paths = sorted(glob.glob(f'{path_to_frames}/*.*g'))
if len(paths)%2: paths = paths[:-1]
frames = [cv2.imread(i) for i in tqdm(paths)]
video = np.array(frames)
_, h, w = video.shape[:3]
print('--- Input ---')
print(f'Frames: {f}, Height: {h}, Width: {w}\n')

new_video = np.swapaxes(video, 0, 2)
new_video = np.swapaxes(new_video, 0, 1)

for i in tqdm(new_video):
  image = cv2.resize(i, (int(w_mult)*orig_w, orig_h))
  name = str(len(glob.glob(f'{path_to_result}/*.*g'))).zfill(7)
  cv2.imwrite(f'{path_to_result}/{name}.png', image)

f, h, w = new_video.shape[:3]
print('--- Output ---')
print(f'Frames: {f}, Height: {h}, Width: {int(w_mult)*orig_w}')
!ffmpeg -y -r $fps_of_video -i /content/result/%07d.png -c:v libx264 -vf fps=$fps_of_video -pix_fmt yuv420p /content/result_z_back.mp4 -hide_banner -loglevel error

100%|██████████| 996/996 [00:01<00:00, 787.47it/s]


--- Input ---
Frames: 1000, Height: 250, Width: 250



100%|██████████| 250/250 [00:04<00:00, 53.54it/s]


--- Output ---
Frames: 250, Height: 250, Width: 1000


In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Upscale height</font></b>
frames_of_video = round(cv2.VideoCapture("/content/result_z_back.mp4").get(cv2.CAP_PROP_FRAME_COUNT))
print('--- Input ---')
print(f'Frames (height): {frames_of_video}\n')

h_mult = "4" #@param ["2", "4"]
exp = str(int(math.log(int(h_mult), 2)))
path_to_output = '/content/rife_h.mp4'

%cd /content/arXiv2020-RIFE/
!python3 inference_video.py --exp=$exp --video /content/result_z_back.mp4 --output $path_to_output --fps=$fps_of_video --scale=1 --skip

!rm -rf /content/frames
!mkdir /content/frames
!ffmpeg -y -i /content/rife_h.mp4 /content/frames/%7d.png -hide_banner -loglevel error

f = int(h_mult)*orig_h
print('\n--- Output ---')
print(f'Frames (height): {f}')

--- Input ---
Frames (height): 250

/content/arXiv2020-RIFE
skip flag is abandoned, please refer to issue #207.
Loaded 3.x/4.x HD model.
/content/result_z_back.mp4, 250.0 frames in total, 25.0FPS to 25FPS
Will not merge audio because using png or fps flag!
100% 249/250.0 [00:06<00:00, 37.25it/s]

--- Output ---
Frames (height): 1000


In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Swap space and time (X-axis back)</font></b>
!rm -rf $path_to_result
!mkdir $path_to_result
paths = sorted(glob.glob(f'{path_to_frames}/*.*g'))
if len(paths)%2: paths = paths[:-1]
frames = [cv2.imread(i) for i in tqdm(paths)]
video = np.array(frames)
_, h, w = video.shape[:3]
print('--- Input ---')
print(f'Frames: {f}, Height: {h}, Width: {int(w_mult)*orig_w}\n')

new_video = np.swapaxes(video, 0, 1)

for i in tqdm(new_video):
  image = cv2.resize(i, (int(w_mult)*orig_w, int(h_mult)*orig_h))
  name = str(len(glob.glob(f'{path_to_result}/*.*g'))).zfill(7)
  cv2.imwrite(f'{path_to_result}/{name}.png', image)

f, h, w = new_video.shape[:3]
print('--- Output ---')
print(f'Frames: {f}, Height: {int(h_mult)*orig_h}, Width: {int(w_mult)*orig_w}')
!ffmpeg -y -r $fps_of_video -i /content/result/%07d.png -c:v libx264 -vf fps=$fps_of_video -pix_fmt yuv420p /content/result_upscaled.mp4 -hide_banner -loglevel error

100%|██████████| 996/996 [00:04<00:00, 235.09it/s]


--- Input ---
Frames: 250, Height: 250, Width: 1000



100%|██████████| 250/250 [00:14<00:00, 16.95it/s]


--- Output ---
Frames: 250, Height: 1000, Width: 1000


In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Download result</font></b>
from google.colab import files
files.download('/content/result_upscaled.mp4')



---

# Real-ESRGAN (frame interpolation)

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Install</font></b>
from IPython.display import clear_output
%cd /content
!rm -rf /content/Real-ESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN
%cd /content/Real-ESRGAN
!git checkout 4211085
!rm -rf /content/Real-ESRGAN/inputs/*
!pip install -U basicsr
!pip install -U facexlib
!python setup.py develop
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P /content/Real-ESRGAN/experiments/pretrained_models
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth -P /content/Real-ESRGAN/experiments/pretrained_models
if os.path.isfile('/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x4plus.pth'):
  clear_output()
  print('Done!')
else: print("Weights didn't download")

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Upload video</font></b>
import os
import cv2
import glob
import math
import numpy as np
from tqdm import tqdm

path_to_frames = '/content/frames'
path_to_result = '/content/result'
video_path = '/content/1_0_moose_hw250_fps25.mp4' #@param {type:"string"}

!rm -rf $path_to_frames
!mkdir $path_to_frames
!ffmpeg -y -i $video_path /content/frames/%7d.png -hide_banner -loglevel error
fps_of_video = round(cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FPS))

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Swap space and time (Z-axis)</font></b>
!rm -rf $path_to_result
!mkdir $path_to_result
paths = sorted(glob.glob(f'{path_to_frames}/*.*g'))
if len(paths)%2: paths = paths[:-1]
frames = [cv2.imread(i) for i in tqdm(paths)]
video = np.array(frames)
f, h, w = video.shape[:3]
print('--- Input ---')
print(f'Frames: {f}, Height: {h}, Width: {w}\n')
orig_h, orig_w = cv2.imread(f'{path_to_frames}/0000001.png').shape[:2]

new_video = np.swapaxes(video, 0, 2)

for i in tqdm(new_video):
  name = str(len(glob.glob(f'{path_to_result}/*.*g'))).zfill(7)
  cv2.imwrite(f'{path_to_result}/{name}.png', i)

f, h, w = new_video.shape[:3]
print('--- Output ---')
print(f'Frames: {f}, Height: {h}, Width: {w}')
!ffmpeg -y -r $fps_of_video -i /content/result/%07d.png -c:v libx264 -vf fps=$fps_of_video -pix_fmt yuv420p /content/result_z.mp4 -hide_banner -loglevel error

100%|██████████| 250/250 [00:00<00:00, 508.49it/s]


--- Input ---
Frames: 250, Height: 250, Width: 250



100%|██████████| 250/250 [00:00<00:00, 322.52it/s]


--- Output ---
Frames: 250, Height: 250, Width: 250


In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Interpolate frames</font></b>
!rm -rf /content/result/.ipynb_checkpoints /content/frames/*
%cd /content/Real-ESRGAN

slowdown_mult = 4 #@param [2, 4]
weight = f'/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x{slowdown_mult}plus.pth'

print(len(glob.glob(f'{path_to_result}/*.*g')))
!python inference_realesrgan.py \
  --model_path $weight \
  --input $path_to_result \
  --output $path_to_frames \
  --netscale $slowdown_mult \
  --outscale $slowdown_mult \
  --tile 512 \
  --tile_pad 20

if len(os.listdir(path_to_result)) == len(os.listdir(path_to_frames)):
  clear_output()
  print('Done!')

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Swap space and time (Z-axis back)</font></b>
!rm -rf $path_to_result
!mkdir $path_to_result
paths = sorted(glob.glob(f'{path_to_frames}/*.*g'))
if len(paths)%2: paths = paths[:-1]
frames = [cv2.imread(i) for i in tqdm(paths)]
video = np.array(frames)
f, h, w = video.shape[:3]
print('--- Input ---')
print(f'Frames: {f}, Height: {h}, Width: {w}\n')

new_video = np.swapaxes(video, 0, 2)

for i in tqdm(new_video):
  image = cv2.resize(i, (orig_w, orig_h))
  name = str(len(glob.glob(f'{path_to_result}/*.*g'))).zfill(7)
  cv2.imwrite(f'{path_to_result}/{name}.png', image)

f, _, w = new_video.shape[:3]
print('--- Output ---')
print(f'Frames: {f}, Height: {orig_h}, Width: {w}')
!ffmpeg -y -r $fps_of_video -i /content/result/%07d.png -c:v libx264 -vf fps=$fps_of_video -pix_fmt yuv420p /content/result_interpol.mp4 -hide_banner -loglevel error

100%|██████████| 250/250 [00:05<00:00, 41.93it/s]


--- Input ---
Frames: 250, Height: 1000, Width: 1000



100%|██████████| 1000/1000 [00:14<00:00, 66.85it/s]


--- Output ---
Frames: 1000, Height: 250, Width: 250


In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Download result</font></b>
from google.colab import files
files.download('/content/result_interpol.mp4')